# Using JAXopt

In this exercise you will use JAXopt to solve a *batched* version of the first and second exercise.

## Resources

- [JAX documentation](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html)
- [jax.numpy documentation](https://jax.readthedocs.io/en/latest/jax.numpy.html)
- [JAXopt documentation](https://jaxopt.github.io/stable/unconstrained.html)

---

Lets start by defining the criterion function from the solutions of the second exercise, but this time we use jax.numpy instead of numpy, and we parametrize the function.

In [1]:
import jax
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)

In [2]:
x0 = {"a": jnp.array(0.0), "b": jnp.zeros(3), "C": jnp.zeros((2, 2))}
b0 = jnp.arange(3, dtype="float64")


def f(x, b0):
    value = (
        (x["a"] - jnp.pi) ** 2
        + jnp.sum((x["b"] - b0) ** 2)
        + jnp.sum((x["C"] - jnp.eye(2)) ** 2)
    )
    return value

## Task 1: Optimize using `JAXopt`

- Create a solver instance of the class `LBFGS` for your problem. You need to make sure to
    - pass the function `f`
    - set `tol=1e-5` for increased accuracy
- Run the optimization using `solver.run`. You need to make sure to
    - pass the initial parameters
    - pass additional arguments of `f`
- Look at the output of the results. How do you access the parameters?

In [3]:
from jaxopt import LBFGS

solver = LBFGS(fun=f, tol=1e-6)

res = solver.run(init_params=x0, b0=b0)

In [4]:
from numpy.testing import assert_array_almost_equal
import numpy as np

assert_array_almost_equal(res.params["a"], np.pi)
assert_array_almost_equal(res.params["b"], np.arange(3))
assert_array_almost_equal(res.params["C"], np.eye(2))

## Task 2: Batched optimization

Now you will optimize `f` not only for a single value of `b`, but for many.

- Write a wrapper for `solver.run` that takes starting values for `x` and a single vector-valued parameter `b0`.
- Use `vmap` and `jit` to create a vectorized and jitted version of this wrapper that allows for array-valued `b0`.
- Execute your vectorized function on `b_arr` (this should perform 500 optimizations)

In [5]:
from jax import jit
from jax import vmap


b_arr = jnp.arange(1500, dtype="float64").reshape(500, 3)


def solve(x, b0):
    return solver.run(init_params=x, b0=b0)

In [6]:
batch_solve = jit(vmap(solve, in_axes=(None, 0)))

In [7]:
result = batch_solve(x0, b_arr)

assert_array_almost_equal(result.params["b"], b_arr)

## Optional: Speed comparison

Lets compare the speed of `jaxopt`'s batch optimization to a loop with scipy's `minimize`.

- Finish the loop in `batch_solve_scipy` using `method="L-BFGS-B"` in scipy's `minimize`
- Time the functions (Use `%timeit func()` in a notebook cell)

In [8]:
import numpy as np
from scipy.optimize import minimize


x0_numpy = np.zeros(8)
b_arr_numpy = np.array(b_arr)


def f_numpy(x, b0):
    a = x[0]
    b = x[1:4]
    C = x[4:].reshape(2, 2)

    value = (a - np.pi) ** 2 + np.sum((b - b0) ** 2) + np.sum((C - np.eye(2)) ** 2)
    return value

In [9]:
def batch_solve_scipy(x0, b_arr):

    results = []
    for b0 in b_arr:
        res = minimize(f_numpy, x0, method="L-BFGS-B", args=(b0,))
        results.append(res.x[1:4])

    return np.stack(results)

In [10]:
results = batch_solve_scipy(x0_numpy, b_arr_numpy)

assert_array_almost_equal(results, b_arr_numpy, decimal=5)

### Timing

In [11]:
%timeit batch_solve(x0, b_arr)

6.74 ms ± 264 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit batch_solve_scipy(x0_numpy, b_arr_numpy)

2.75 s ± 242 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
